ResNet Testing

In [13]:
import torch
from resnet import ResNet18
from collections import OrderedDict

In [2]:
torch.cuda.is_available()

True

In [20]:
net = ResNet18()
checkpoint = torch.load('./checkpoint/ckpt.pth')

# This file does not use DataParallel, using the soltion shown here
# https://discuss.pytorch.org/t/solved-keyerror-unexpected-key-module-encoder-embedding-weight-in-state-dict/1686/2

new_state_dict = OrderedDict()
for k, v in checkpoint['net'].items():
    name = k[7:] # this removes the 'module.' from the front (notice it has 7 characters)
    new_state_dict[name] = v

In [22]:
# Moment of truth, what happens???
net.load_state_dict(new_state_dict)

<All keys matched successfully>

In [21]:
checkpoint['acc']

95.47